# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), True)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# You sit North and hold:
hand = 'J.32.KJ983.QJT74'

auction = ["PAD_START","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 0, True)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Setting seed (Sampling bidding info) from J.32.KJ983.QJT74: 2930407879
bid X value 0.6273 is recommended by NN
CandidateBid(bid=X   , insta_score=0.6273, expected_score=---, adjust=---)
CandidateBid(bid=PASS, insta_score=0.3717, expected_score=---, adjust=---)
Sampling for aution: ['PAD_START', '2C'] trying to find 5000
sample_cards_auction, nsteps= 1
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', '2C']
nesw_i 2
bids in model 3
Player:  S Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 6.4   4.88 21.  ]]
Shape:  [[3.94 3.68 2.76 2.56 3.72 3.52 2.91 2.79 4.09 3.69 2.76 2.41]]
sample_cards_vec generating 5000
sample_cards_vec took 0.0987 Deals: 2977
n_samples 2977 matching bidding info
Samples after bidding filtering:  2977  Threshold:  0.7
Found 2977 samples for bidding
 X    0.627 Samples: 200
bidding_rollout - n_samples:  200
bidding_rollout - finished  (200, 64)


Loaded lib dds.dll


dds took 2.347
[  100.   200.  -650.  -680.  -620.  -170.   100.   200.   100.  -710.  -680.  -650.  -650. -1430.  -680.   200. -1460.   200.  -710.   200.   100.   200.  -620.   100.   100.   100.  -650.  -200.
   300.  -620.   100.  -630.  -200.   100.  -650.  -620.   100.  -680.  -630.   100.   200.  -620.  -620.  -650.  -620.  -660.   200.  -710.   300.  -650.   200.   100.   300. -1460.   300.  -680.
   100.  -650.  -650.   200.   100.  -710.  -620.   100.   100. -1460. -1460.  -200.  -650.   100.   100.  -600. -1440.  -200.  -650.  -630.   100.  -620.  -650.  -620.   300.   100.  -650.   100.
   100.  -650.  -680.   300.  -710.   100. -1460.  -680.   400.  -650.   300.   200.  -620. -1430. -1460.   200.   100. -1430.  -600.  -650.  -690. -1100.  -710.  -710.  -650.  -210.  -620.   300.
  -650.  1100.  -200.   100.  -630.   100.   100.  -680.   200.   300.   200.   100.  -650.   100.  -200.  -630.  -630.  -620.   200.   100.  -650.  -630.  -620.  -620.   100.  -620.  -630.  -650.


[{'call': 'PASS',
  'insta_score': 0.372,
  'expected_score': -258,
  'adjustment': 19},
 {'call': 'X', 'insta_score': 0.627, 'expected_score': -370, 'adjustment': 31}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


98xx.Txx.AQTx.xx AKTx.AQ98.xx.A9x J.xx.KJ98x.QJTxx Qxxx.KJxx.xx.K8x 0.75000 
 2C-X-2N-P-3N-P-P-P (8, 7.5, 10.0)  
 2C-P-2N-P-3N-P-4N-P-5C-P-P-P (5, 6.8, 9.6) 
Axx.Q8xx.Txx.K9x Q98x.AKxx.AQ.Axx J.xx.KJ98x.QJTxx KTxxx.JT9.xxx.8x 0.75009 
 2C-X-2D-P-2H-P-3C-P-3H-P-4H-P-P-P (8, 8.2, 8.5)  
 2C-P-2D-P-2H-P-3H-P-4N-P-5D-P-5H-P-P-P (8, 8.2, 8.5) 
K9xx.QJ.QTx.9xxx AQ8xx.8xx.Axx.AK J.xx.KJ98x.QJTxx Txx.AKT9xx.xx.8x 0.75001 
 2C-X-2H-P-4H-P-P-P (11, 10.6, 11.0)  
 2C-P-2H-P-4H-P-P-P (11, 10.6, 11.0) 
9xxx.QT.Qxxx.A9x AQT.AJ8xx.A.K8xx J.xx.KJ98x.QJTxx K8xxx.K9xx.Txx.x 0.75012 
 2C-X-2D-P-3C-P-3S-P-4H-P-P-P (12, 11.4, 11.3)  
 2C-P-2D-P-3C-P-3S-P-4C-P-P-P (7, 5.8, 8.9) 
KTxx.QJx.Qxx.8xx A98x.K8x.ATx.AK9 J.xx.KJ98x.QJTxx Qxxx.AT9xx.xx.xx 0.75001 
 2C-X-2D-P-2H-P-3C-P-3H-P-4H-P-P-P (10, 9.8, 9.3)  
 2C-P-2D-P-2H-P-3H-P-4N-P-5D-P-5H-P-P-P (10, 9.8, 9.3) 
8xxx.AK8xx.T.9xx AQx.QT9x.Axxx.AK J.xx.KJ98x.QJTxx KT9xx.Jx.Qxx.8xx 0.75003 
 2C-X-2D-2H-P-P-2S-P-P-P (10, 9.7, 9.5)  
 2C-P-2D-P-2H-P-2S-P-4N-P-5D-

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,0)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

sample_cards_auction, nsteps= 3
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', '1N', 'PASS', '3S', 'PASS']
nesw_i 0
bids in model 3
Player:  N Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 7.98 16.72  7.62]]
Shape:  [[2.58 4.16 3.25 3.06 6.29 2.82 1.93 1.87 2.74 3.91 3.21 3.13]]
sample_cards_vec generating 5000
sample_cards_vec took 0.09999 Deals: 2434
n_samples 5000 matching bidding info
Samples after bidding filtering:  0  Threshold:  0.7
Only found 2167 10
Found 10 samples for bidding
Good quality: True
J.xx.KJ98x.QJTxx Kxxx.JT8xx.T.8xx T98x.AKQ.Axx.AKx AQxx.9xx.Qxxx.9x 0.50
J.xx.KJ98x.QJTxx ATxx.JTx.Txx.Axx Q9xx.AKx.Ax.K98x K8xx.Q98xx.Qxx.x 0.50
J.xx.KJ98x.QJTxx ATxx.98xx.T.K98x Kxxx.AQJx.AQx.Ax Q98x.KTx.xxxx.xx 0.50
J.xx.KJ98x.QJTxx Qxxx.Tx.Qxxx.K9x AKTx.AJ8x.A.A8xx 98xx.KQ9xx.Txx.x 0.50
J.xx.KJ98x.QJTxx Axxx.Kxxx.Txx.xx K98x.QJ9x.AQx.AK QTxx.AT8.xx.98xx 0.50
J.xx.KJ98x.QJTxx KQTxx.Txx.x.98xx A9xx.AKQx.Axxx.A 8xx.J98x.QTx.Kxx 0.50
J.xx.KJ98x.QJTxx Kxxx.9x.xxx.A9xx